# Multirate Kalman Filter Design via LMI Optimization
# Automotive Navigation System

**Author:** Hiroshi Okajima  
**Date:** February 2025

## Description
This notebook implements a multirate Kalman filter for automotive navigation systems using LMI-based optimization through cyclic reformulation.

The system fuses:
- **GPS measurements** (1 Hz) - position only
- **Wheel speed sensor data** (10 Hz) - velocity

to estimate vehicle **position**, **velocity**, and **acceleration**.

## System Configuration
- State: $x = [\text{position}; \text{velocity}; \text{acceleration}]$ (n=3)
- Measurements: GPS position (10% availability) + wheel speed (100%) (m=2)
- Period: N = 10, Sampling time: dt = 0.1s

## Reference
H. Okajima, "LMI Optimization Based Multirate Steady-State Kalman Filter Design," IEEE Access, 2025.

## 0. Install Required Packages

In [ ]:
!pip install numpy scipy matplotlib cvxpy

## 1. Import Libraries

In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm

# For better plot display in notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [14, 10]
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"CVXPY version: {cp.__version__}")

## 2. System Definition

### Vehicle Dynamics Model
$$x(k+1) = Ax(k) + Bu(k) + w(k), \quad w \sim \mathcal{N}(0, Q)$$
$$y(k) = S_k C x(k) + S_k v(k), \quad v \sim \mathcal{N}(0, R)$$

where $S_k$ is the time-varying sensor availability matrix.

In [ ]:
# System parameters
n = 3      # State dimension [position, velocity, acceleration]
p = 1      # Input dimension
m = 2      # Output dimension [GPS position, wheel speed]
N = 10     # Period (GPS: 1Hz, Wheel speed: 10Hz)
dt = 0.1   # Sampling time [s]

# State-space model: x = [position; velocity; acceleration]
A = np.array([
    [1,  dt,  0.5*dt**2],
    [0,  1,   dt],
    [0,  0,   0.8]
])

B = np.array([[0], [0], [1]])

# Output: position and velocity
C = np.array([
    [1, 0, 0],   # GPS position
    [0, 1, 0]    # Wheel speed sensor
])

# Noise covariances
Q = np.diag([0.01, 0.1, 0.5])    # Process noise
R = np.diag([1.0, 0.1])          # GPS ±1m, wheel speed ±0.3m/s

print("="*60)
print("Automotive Navigation System")
print("="*60)
print(f"State dimension: n = {n} [position, velocity, acceleration]")
print(f"Output dimension: m = {m} [GPS position, wheel speed]")
print(f"Period: N = {N}")
print(f"Sampling time: dt = {dt} [s]")
print(f"\nEigenvalues of A: {np.linalg.eigvals(A)}")
print(f"\nSensor Specifications:")
print(f"  GPS: 1Hz (every {N} steps), accuracy ±{np.sqrt(R[0,0]):.1f}m")
print(f"  Wheel speed: 10Hz (every step), accuracy ±{np.sqrt(R[1,1]):.1f}m/s")

## 3. Measurement Pattern

- $k \mod 10 = 0$: GPS **ON** + Wheel speed **ON**
- $k \mod 10 = 1,\ldots,9$: GPS **OFF** + Wheel speed **ON**

In [ ]:
# Measurement pattern S_k
S = []
S.append(np.diag([1, 1]))  # k mod 10 = 0: GPS + wheel speed
for i in range(1, N):
    S.append(np.diag([0, 1]))  # k mod 10 = 1-9: wheel speed only

print("Measurement Pattern:")
print(f"  S_0 = diag({np.diag(S[0]).tolist()}) - GPS ON + Wheel speed ON")
print(f"  S_1 = diag({np.diag(S[1]).tolist()}) - GPS OFF + Wheel speed ON")
print(f"  ...")
print(f"  S_9 = diag({np.diag(S[9]).tolist()}) - GPS OFF + Wheel speed ON")

# Visualize measurement pattern
fig, ax = plt.subplots(1, 1, figsize=(12, 3))
pattern = np.zeros((2, N))
for k in range(N):
    pattern[:, k] = np.diag(S[k])
ax.imshow(pattern, cmap='Greens', aspect='auto')
ax.set_yticks([0, 1])
ax.set_yticklabels(['GPS (position)', 'Wheel speed (velocity)'])
ax.set_xticks(range(N))
ax.set_xticklabels([f'k={i}' for i in range(N)])
ax.set_xlabel('Time step (k mod N)')
ax.set_title('Sensor Availability Pattern')
for i in range(2):
    for j in range(N):
        ax.text(j, i, 'ON' if pattern[i,j] else 'OFF', ha='center', va='center', fontsize=9)
plt.tight_layout()
plt.show()

## 4. Cyclic Reformulation (Section III)

### Cyclic System Matrices (Equations 24-28)

$$\check{A} = \begin{bmatrix} 0 & 0 & \cdots & 0 & A \\ A & 0 & \cdots & 0 & 0 \\ 0 & A & \cdots & 0 & 0 \\ \vdots & & \ddots & & \vdots \\ 0 & 0 & \cdots & A & 0 \end{bmatrix}, \quad
\check{C} = \text{diag}(S_0 C, S_1 C, \ldots, S_{N-1} C)$$

In [ ]:
def construct_cyclic_system(A, B, C, Q, R, S, N):
    """Construct cyclic reformulation (Section III)"""
    n = A.shape[0]
    p = B.shape[1]
    m = C.shape[0]
    
    n_cyc = N * n
    m_cyc = N * m
    p_cyc = N * p
    
    # A_cyc: cyclic structure (Equation 24)
    A_cyc = np.zeros((n_cyc, n_cyc))
    A_cyc[0:n, (N-1)*n:N*n] = A
    for i in range(1, N):
        A_cyc[i*n:(i+1)*n, (i-1)*n:i*n] = A
    
    # B_cyc: cyclic structure
    B_cyc = np.zeros((n_cyc, p_cyc))
    B_cyc[0:n, (N-1)*p:N*p] = B
    for i in range(1, N):
        B_cyc[i*n:(i+1)*n, (i-1)*p:i*p] = B
    
    # C_cyc: block-diagonal (Equation 25)
    C_cyc = np.zeros((m_cyc, n_cyc))
    for i in range(N):
        C_cyc[i*m:(i+1)*m, i*n:(i+1)*n] = S[i] @ C
    
    # Q_cyc, R_cyc (Equations 26-27)
    Q_cyc = np.kron(np.eye(N), Q)
    R_cyc = np.zeros((m_cyc, m_cyc))
    for i in range(N):
        R_cyc[i*m:(i+1)*m, i*m:(i+1)*m] = S[i] @ R @ S[i].T
    
    return A_cyc, B_cyc, C_cyc, Q_cyc, R_cyc

A_cyc, B_cyc, C_cyc, Q_cyc, R_cyc = construct_cyclic_system(A, B, C, Q, R, S, N)

n_cyc = N * n
m_cyc = N * m

print("Cyclic System Dimensions:")
print(f"  A_cyc: {A_cyc.shape}")
print(f"  B_cyc: {B_cyc.shape}")
print(f"  C_cyc: {C_cyc.shape}")
print(f"  Q_cyc: {Q_cyc.shape}")
print(f"  R_cyc: {R_cyc.shape} (rank: {np.linalg.matrix_rank(R_cyc)})")
print(f"\nR_cyc is SEMIDEFINITE (rank {np.linalg.matrix_rank(R_cyc)} < {m_cyc})")
print("→ Standard DARE cannot be used, LMI approach is required!")

In [ ]:
# Visualize cyclic matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

ax1 = axes[0]
ax1.spy(A_cyc, markersize=3)
ax1.set_title('$\\check{A}$ (Cyclic Structure)')
ax1.set_xlabel('Column')
ax1.set_ylabel('Row')

ax2 = axes[1]
ax2.spy(C_cyc, markersize=3)
ax2.set_title('$\\check{C}$ (Block Diagonal)')
ax2.set_xlabel('Column')
ax2.set_ylabel('Row')

ax3 = axes[2]
ax3.spy(R_cyc, markersize=5)
ax3.set_title('$\\check{R}$ (Semidefinite)')
ax3.set_xlabel('Column')
ax3.set_ylabel('Row')

plt.suptitle('Cyclic System Matrix Structures', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Observability Check (Proposition 2)

In [ ]:
def check_observability(A_cyc, C_cyc):
    """Check observability of cyclic system"""
    n_cyc = A_cyc.shape[0]
    O = C_cyc.copy()
    Ak = np.eye(n_cyc)
    for i in range(1, n_cyc):
        Ak = Ak @ A_cyc
        O = np.vstack([O, C_cyc @ Ak])
    return np.linalg.matrix_rank(O), np.linalg.matrix_rank(O) == n_cyc

rank_O, is_obs = check_observability(A_cyc, C_cyc)
print(f"Observability matrix rank: {rank_O} / {n_cyc}")
print(f"Result: {'Observable ✓' if is_obs else 'NOT Observable ✗'}")

## 6. Construct $\check{Q}^{1/2}$ and $\check{R}^{1/2}$

In [ ]:
def construct_sqrt_matrices(Q, R, S, N):
    """Construct Q_cyc^{1/2} and R_cyc^{1/2}"""
    n = Q.shape[0]
    m = R.shape[0]
    n_cyc = N * n
    m_cyc = N * m
    
    Q_sqrt = sqrtm(Q).real
    R_sqrt = sqrtm(R).real
    
    # Q_cyc_sqrt: cyclic structure
    Q_cyc_sqrt = np.zeros((n_cyc, n_cyc))
    Q_cyc_sqrt[0:n, (N-1)*n:N*n] = Q_sqrt
    for i in range(1, N):
        Q_cyc_sqrt[i*n:(i+1)*n, (i-1)*n:i*n] = Q_sqrt
    
    # R_cyc_sqrt: block diagonal
    R_cyc_sqrt = np.zeros((m_cyc, m_cyc))
    for i in range(N):
        R_cyc_sqrt[i*m:(i+1)*m, i*m:(i+1)*m] = S[i] @ R_sqrt
    
    return Q_cyc_sqrt, R_cyc_sqrt

Q_cyc_sqrt, R_cyc_sqrt = construct_sqrt_matrices(Q, R, S, N)
print(f"Q_cyc_sqrt: {Q_cyc_sqrt.shape}")
print(f"R_cyc_sqrt: {R_cyc_sqrt.shape}")

## 7. LMI-based Filter Design (Section IV, Equation 39)

### LMI Formulation

Decision variables:
- $\check{X} \in \mathbb{R}^{Nn \times Nn}$: $\check{X} \succ 0$ (Lyapunov matrix, $\check{X} = \check{P}^{-1}$)
- $\check{Y} \in \mathbb{R}^{Nn \times Nq}$: $\check{Y} = -\check{X}\check{L}$ (gain variable)

$$\begin{bmatrix} \check{X} & \check{X}\check{A} + \check{Y}\check{C} & \check{X}\check{Q}^{1/2} & \check{Y}\check{R}^{1/2} \\ (\check{X}\check{A} + \check{Y}\check{C})^T & \check{X} & 0 & 0 \\ (\check{Q}^{1/2})^T\check{X} & 0 & I_{Nn} & 0 \\ (\check{R}^{1/2})^T\check{Y}^T & 0 & 0 & I_{Nq} \end{bmatrix} \succeq 0$$

### Kalman Gain Extraction (Equation 43)
$$\check{L} = -\check{X}^{-1}\check{Y}$$

In [ ]:
def solve_lmi_kalman_filter(A_cyc, C_cyc, Q_cyc_sqrt, R_cyc_sqrt, epsilon=1e-6):
    """Solve LMI optimization for multirate Kalman filter (Equation 39)"""
    n_cyc = A_cyc.shape[0]
    m_cyc = C_cyc.shape[0]
    
    # Decision variables (matching paper notation)
    # X = P^{-1}, Y = -X*L
    X = cp.Variable((n_cyc, n_cyc), symmetric=True)
    Y = cp.Variable((n_cyc, m_cyc))
    W = cp.Variable((n_cyc, n_cyc), symmetric=True)
    
    constraints = []
    
    # LMI #1 (Equation 39)
    # (1,2): X*A + Y*C
    # (3,1): (Q^{1/2})^T * X
    # (4,1): (R^{1/2})^T * Y^T
    block_12 = X @ A_cyc + Y @ C_cyc
    block_31 = Q_cyc_sqrt.T @ X
    block_41 = R_cyc_sqrt.T @ Y.T
    
    LMI1 = cp.bmat([
        [X, block_12, block_31.T, block_41.T],
        [block_12.T, X, np.zeros((n_cyc, n_cyc)), np.zeros((n_cyc, m_cyc))],
        [block_31, np.zeros((n_cyc, n_cyc)), np.eye(n_cyc), np.zeros((n_cyc, m_cyc))],
        [block_41, np.zeros((m_cyc, n_cyc)), np.zeros((m_cyc, n_cyc)), np.eye(m_cyc)]
    ])
    constraints.append(LMI1 >> 0)
    constraints.append(X >> epsilon * np.eye(n_cyc))
    
    # LMI #2 (Equation 41): W >= X^{-1} = P
    LMI2 = cp.bmat([[W, np.eye(n_cyc)], [np.eye(n_cyc), X]])
    constraints.append(LMI2 >> 0)
    
    # Objective (Equation 42): minimize trace(W)
    problem = cp.Problem(cp.Minimize(cp.trace(W)), constraints)
    problem.solve(solver=cp.SCS, verbose=False, max_iters=10000, eps=1e-9)
    
    if problem.status not in ['optimal', 'optimal_inaccurate']:
        raise ValueError(f"LMI failed: {problem.status}")
    
    X_opt, Y_opt, W_opt = X.value, Y.value, W.value
    P_ss = np.linalg.inv(X_opt)
    
    # Kalman gain (Equation 43): L = -X^{-1} Y
    L_cyc = -np.linalg.solve(X_opt, Y_opt)
    
    return X_opt, Y_opt, W_opt, L_cyc, P_ss, problem.value

print("Solving LMI optimization...")
X_opt, Y_opt, W_opt, L_cyc, P_ss, opt_trace = solve_lmi_kalman_filter(
    A_cyc, C_cyc, Q_cyc_sqrt, R_cyc_sqrt
)

print(f"\nLMI Results:")
print(f"  Optimal trace(W) = {opt_trace:.6f}")
print(f"  Actual trace(P) = {np.trace(P_ss):.6f}")
print(f"  L_cyc size: {L_cyc.shape}")

In [ ]:
# Stability check
A_cl = A_cyc - L_cyc @ C_cyc
eig_cl = np.linalg.eigvals(A_cl)
max_eig = np.max(np.abs(eig_cl))

print("Stability Analysis:")
print(f"  Max |eigenvalue| of A_cyc - L_cyc*C_cyc: {max_eig:.10f}")
print(f"  Result: {'STABLE ✓' if max_eig < 1 else 'UNSTABLE ✗'}")

## 8. Extract Periodic Kalman Gains (Equation 44)

In [ ]:
def extract_periodic_gains(L_cyc, N, n, m):
    """Extract periodic gains from L_cyc (Equation 44)"""
    L = []
    # L_0 to L_{N-2}
    for k in range(N-1):
        L.append(L_cyc[(k+1)*n:(k+2)*n, k*m:(k+1)*m])
    # L_{N-1}
    L.append(L_cyc[0:n, (N-1)*m:N*m])
    return L

L = extract_periodic_gains(L_cyc, N, n, m)

print("Periodic Kalman Gains:")
print(f"\nL_0 (k mod {N} = 0, GPS + wheel speed):")
print(L[0])
print(f"\nL_1 (k mod {N} = 1, wheel speed only):")
print(L[1])
print(f"\nL_5 (k mod {N} = 5, wheel speed only):")
print(L[5])

In [ ]:
# Visualize periodic gains
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gain norms
ax1 = axes[0]
L_norms = [np.linalg.norm(L[k], 'fro') for k in range(N)]
colors = ['green' if k == 0 else 'blue' for k in range(N)]
ax1.bar(range(N), L_norms, color=colors)
ax1.set_xlabel('k mod N')
ax1.set_ylabel('||L_k|| (Frobenius)')
ax1.set_title('Periodic Kalman Gain Norms')
ax1.set_xticks(range(N))
ax1.grid(True, alpha=0.3)

# Gain contributions
ax2 = axes[1]
L_contributions = np.zeros((N, n))
for k in range(N):
    for j in range(n):
        L_contributions[k, j] = np.linalg.norm(L[k][j, :])

x = np.arange(N)
width = 0.25
ax2.bar(x - width, L_contributions[:, 0], width, label='Position')
ax2.bar(x, L_contributions[:, 1], width, label='Velocity')
ax2.bar(x + width, L_contributions[:, 2], width, label='Acceleration')
ax2.set_xlabel('k mod N')
ax2.set_ylabel('Gain Magnitude')
ax2.set_title('Kalman Gain Contribution to Each State')
ax2.set_xticks(range(N))
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Simulation

In [ ]:
# Simulation parameters
T = 200    # Simulation length
np.random.seed(42)

# Initial state
x0 = np.array([0, 5, 0])  # position 0m, velocity 5m/s

# True state
x_true = np.zeros((n, T))
x_true[:, 0] = x0

# Observations
z_obs = np.zeros((m, T))

# Control input
u = 0.5 * np.sin(0.05 * np.arange(T))

# Generate true trajectory
Q_sqrt = sqrtm(Q).real
R_sqrt = sqrtm(R).real

for k in range(T - 1):
    w = Q_sqrt @ np.random.randn(n)
    x_true[:, k+1] = A @ x_true[:, k] + B.flatten() * u[k] + w

# Generate observations
for k in range(T):
    v = R_sqrt @ np.random.randn(m)
    z_obs[:, k] = C @ x_true[:, k] + v

print(f"Simulation:")
print(f"  Total time: {T*dt:.1f} seconds")
print(f"  Total distance: {x_true[0, -1]:.1f} m")
print(f"  Max velocity: {np.max(x_true[1, :]):.1f} m/s ({np.max(x_true[1, :])*3.6:.1f} km/h)")

In [ ]:
# Kalman filter estimation
x_hat = np.zeros((n, T))
x_hat[:, 0] = x_true[:, 0]  # Perfect initial condition

for k in range(T - 1):
    # Prediction
    x_pred = A @ x_hat[:, k] + B.flatten() * u[k]
    z_pred = C @ x_pred
    
    # Update with periodic gain
    idx = k % N
    innovation = z_obs[:, k+1] - z_pred
    x_hat[:, k+1] = x_pred + L[idx] @ innovation

print("Kalman filter completed!")

## 10. Performance Evaluation

In [ ]:
error = x_true - x_hat
rmse = np.sqrt(np.mean(error**2, axis=1))
max_error = np.max(np.abs(error), axis=1)

print("Performance Metrics:")
print(f"\nRMSE:")
print(f"  Position:     {rmse[0]:.4f} [m]")
print(f"  Velocity:     {rmse[1]:.4f} [m/s]")
print(f"  Acceleration: {rmse[2]:.4f} [m/s²]")

print(f"\nMaximum Error:")
print(f"  Position:     {max_error[0]:.4f} [m]")
print(f"  Velocity:     {max_error[1]:.4f} [m/s]")
print(f"  Acceleration: {max_error[2]:.4f} [m/s²]")

## 11. Visualization

In [ ]:
fig = plt.figure(figsize=(16, 12))

# Position
ax1 = fig.add_subplot(3, 3, 1)
ax1.plot(range(T), x_true[0, :], 'k-', lw=2, label='True')
ax1.plot(range(T), x_hat[0, :], 'b--', lw=1.5, label='Estimated')
obs_idx = [k for k in range(T) if k % N == 0]
ax1.plot(obs_idx, z_obs[0, obs_idx], 'go', ms=5, label='GPS')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Position [m]')
ax1.set_title('Position (GPS: every 10 steps)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Velocity
ax2 = fig.add_subplot(3, 3, 2)
ax2.plot(range(T), x_true[1, :], 'k-', lw=2, label='True')
ax2.plot(range(T), x_hat[1, :], 'b--', lw=1.5, label='Estimated')
ax2.set_xlabel('Time Step')
ax2.set_ylabel('Velocity [m/s]')
ax2.set_title('Velocity (Wheel Speed: every step)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Acceleration
ax3 = fig.add_subplot(3, 3, 3)
ax3.plot(range(T), x_true[2, :], 'k-', lw=2, label='True')
ax3.plot(range(T), x_hat[2, :], 'b--', lw=1.5, label='Estimated')
ax3.set_xlabel('Time Step')
ax3.set_ylabel('Acceleration [m/s²]')
ax3.set_title('Acceleration (not observed)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Estimation errors
ax4 = fig.add_subplot(3, 3, 4)
ax4.plot(range(T), error[0, :], lw=1.5, label='Position')
ax4.plot(range(T), error[1, :], lw=1.5, label='Velocity')
ax4.plot(range(T), error[2, :], lw=1.5, label='Acceleration')
ax4.axhline(0, color='k', ls='--')
ax4.set_xlabel('Time Step')
ax4.set_ylabel('Error')
ax4.set_title('Estimation Errors')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Position error
ax5 = fig.add_subplot(3, 3, 5)
ax5.plot(range(T), error[0, :], lw=1.5)
ax5.axhline(0, color='k', ls='--')
ax5.set_xlabel('Time Step')
ax5.set_ylabel('Position Error [m]')
ax5.set_title(f'Position Error (RMSE={rmse[0]:.4f} m)')
ax5.grid(True, alpha=0.3)

# Velocity error
ax6 = fig.add_subplot(3, 3, 6)
ax6.plot(range(T), error[1, :], lw=1.5)
ax6.axhline(0, color='k', ls='--')
ax6.set_xlabel('Time Step')
ax6.set_ylabel('Velocity Error [m/s]')
ax6.set_title(f'Velocity Error (RMSE={rmse[1]:.4f} m/s)')
ax6.grid(True, alpha=0.3)

# Closed-loop eigenvalues
ax7 = fig.add_subplot(3, 3, 7)
theta = np.linspace(0, 2*np.pi, 100)
ax7.plot(np.cos(theta), np.sin(theta), 'k--', lw=1)
ax7.plot(np.real(eig_cl), np.imag(eig_cl), 'bo', ms=6)
ax7.set_aspect('equal')
ax7.set_xlabel('Real')
ax7.set_ylabel('Imaginary')
ax7.set_title(f'Closed-loop Eigenvalues (max|λ|={max_eig:.4f})')
ax7.grid(True, alpha=0.3)

# Eigenvalue magnitudes
ax8 = fig.add_subplot(3, 3, 8)
ax8.bar(range(len(eig_cl)), np.abs(eig_cl))
ax8.axhline(1.0, color='k', ls='--', lw=2)
ax8.set_xlabel('Index')
ax8.set_ylabel('|λ|')
ax8.set_title('Eigenvalue Magnitudes')
ax8.set_ylim([0, 1.1])
ax8.grid(True, alpha=0.3)

# P matrix eigenvalues
ax9 = fig.add_subplot(3, 3, 9)
P_eigs = np.sort(np.linalg.eigvals(P_ss).real)[::-1]
ax9.semilogy(range(1, len(P_eigs)+1), P_eigs, 'bo-', lw=1.5)
ax9.set_xlabel('Index')
ax9.set_ylabel('Eigenvalue')
ax9.set_title(f'P Matrix Eigenvalues (trace={np.trace(P_ss):.2f})')
ax9.grid(True, alpha=0.3)

plt.suptitle('Multirate Kalman Filter: Automotive Navigation (n=3, m=2, N=10)', fontsize=14)
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated LMI-based multirate Kalman filter design for automotive navigation.

**Key Points:**
1. GPS (1Hz) and wheel speed (10Hz) sensors have different sampling rates
2. $\check{R}$ is semidefinite when GPS is unavailable (standard DARE fails)
3. LMI approach handles semidefinite $\check{R}$ naturally
4. Kalman gain (Equation 43): $\check{L} = -\check{X}^{-1}\check{Y}$

**Performance:**
- Position RMSE: ~0.4 m (with GPS every 10 steps)
- Velocity RMSE: ~0.2 m/s (with continuous wheel speed)
- Acceleration RMSE: ~1.3 m/s² (not directly observed)